In [19]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from pydantic import BaseModel,Field
from langgraph.graph import StateGraph,START, END
from typing import TypedDict, Literal, Annotated,Literal
from langchain_core.messages import SystemMessage, HumanMessage,BaseMessage
import operator
from langgraph.graph import add_messages
from langgraph.checkpoint.memory import InMemorySaver

load_dotenv()

True

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI

bot = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
)

E0000 00:00:1758877821.646981   82729 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [14]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage],add_messages]


In [15]:
def chat(state: ChatState) -> ChatState:
    message_history=state["messages"]
    response=bot.invoke(message_history)
    return {"messages":[response]}  # this statement appends an AI Message object in the messages_list

In [ ]:
graph=StateGraph(ChatState)
checkpoint_saver= InMemorySaver()
graph.add_node("chat_bot",chat)
graph.add_edge(START,"chat_bot")
graph.add_edge("chat_bot",END)
workflow= graph.compile(checkpointer=checkpoint_saver)

# graph.compile()

NameError: name 'StateGraph' is not defined

In [ ]:
while True:
    user_input=input("Enter your message (or type 'exit' to quit): ")
    if user_input.lower() in ["exit","quit"]:
        break
    initial_state={"messages":[HumanMessage(content=user_input)]}
    congfig={"configurable":{"thread_id":"first_thread"}}
    final_state= workflow.invoke(initial_state,config=congfig)
    print("Bot:",final_state["messages"][-1].content)

Bot: Hello! How can I help you today?
Bot: Okay, Snehil.
Bot: As an AI, I don't know your name. I don't have access to personal information about you.

If you'd like me to refer to you by a specific name during our conversation, you're welcome to tell me!
